<h1 style="text-align: center;" markdown="1">Random Forest Model - 4</h1>

We tried to build final model. We used feature created for model -2, important features in model-1 and 
new features created using combination of Jaccard similarity and Hierarchial Clustering

We got 8% accuracy using this technique

We had downsampled data significantly. I think, larger sample size could have added to the accuracy of this model

<h2 style="text-align: center;" markdown="1">Loading and Reading Data </h2>

In [2]:
import pandas as pd

### Configure path and load the data

In [2]:
#### Configure path
dataRFPath = "/home/smita/RF_models/dataRF4.csv"

In [3]:
full_df14 = pd.read_csv(dataRFPath)

In [4]:
full_df14.shape

(26483412, 28)

In [5]:
full_df14.iloc[0:3, 0:12]

,Unnamed: 0,srch_destination_id,hotel_cluster,user_id,is_mobile,is_package,is_booking,is_alone,hotel_continent,hotel_country,hotel_market,orig_destination_distance
0,1,1,30,1195632,0,0,0,0.0,5,76,1537,NaN
1,2,1,60,1195632,0,1,0,0.0,5,76,1537,NaN
2,3,1,30,999935,0,0,0,0.0,5,76,1537,NaN


In [6]:
full_df14.iloc[0:3, 13:28]

,srch_rm_cnt,user_location_country,user_location_region,user_location_city,year,month,day,hour,part_of_day,type_of_day,season,duration_of_stay,dest_feature_pc1,dest_feature_pc2,dest_feature_pc3
0,6,231,48,2982,2014,9,10,13,evening,weekday,fall,1.0,-6.605425,-0.249541,0.599817
1,6,231,48,2982,2014,8,28,13,evening,weekday,summer,1.0,-6.605425,-0.249541,0.599817
2,2,1,401,230,2014,2,7,9,evening,weekday,winter,1.0,-6.605425,-0.249541,0.599817


In [7]:
full_df14.columns.values

array(['Unnamed: 0', 'srch_destination_id', 'hotel_cluster', 'user_id',
       'is_mobile', 'is_package', 'is_booking', 'is_alone',
       'hotel_continent', 'hotel_country', 'hotel_market',
       'orig_destination_distance', 'srch_destination_type_id',
       'srch_rm_cnt', 'user_location_country', 'user_location_region',
       'user_location_city', 'year', 'month', 'day', 'hour', 'part_of_day',
       'type_of_day', 'season', 'duration_of_stay', 'dest_feature_pc1',
       'dest_feature_pc2', 'dest_feature_pc3'], dtype=object)

In [8]:
### Remove index columns from the data : Unnamed: 0 
del full_df14['Unnamed: 0']

In [9]:
full_df14.columns.values

array(['srch_destination_id', 'hotel_cluster', 'user_id', 'is_mobile',
       'is_package', 'is_booking', 'is_alone', 'hotel_continent',
       'hotel_country', 'hotel_market', 'orig_destination_distance',
       'srch_destination_type_id', 'srch_rm_cnt', 'user_location_country',
       'user_location_region', 'user_location_city', 'year', 'month',
       'day', 'hour', 'part_of_day', 'type_of_day', 'season',
       'duration_of_stay', 'dest_feature_pc1', 'dest_feature_pc2',
       'dest_feature_pc3'], dtype=object)

<h2 style="text-align: center;" markdown="1">Data Cleaning </h2>

### Missing Values

In [10]:
count_cols = full_df14.count(axis=0)

In [17]:
count_cols.sort_values()

orig_destination_distance    16976923
dest_feature_pc3             26350093
dest_feature_pc1             26350093
dest_feature_pc2             26350093
is_alone                     26439312
duration_of_stay             26471564
hotel_cluster                26483412
user_id                      26483412
is_mobile                    26483412
is_package                   26483412
is_booking                   26483412
hotel_continent              26483412
hotel_country                26483412
hotel_market                 26483412
srch_destination_type_id     26483412
user_location_city           26483412
srch_destination_id          26483412
user_location_region         26483412
year                         26483412
month                        26483412
day                          26483412
hour                         26483412
part_of_day                  26483412
type_of_day                  26483412
season                       26483412
srch_rm_cnt                  26483412
user_locatio

### Imputing missing values

In [18]:
full_df14['is_alone'].value_counts()

0.0    21466123
1.0     4973189
Name: is_alone, dtype: int64

In [19]:
### It's obvious there should be atleast one person , so we will impute missing value as 1: true
full_df14['is_alone'].fillna(1 ,inplace=True)

In [20]:
### we will impute -1 for principal components : as we don't have pc features for all serch_dest_id
full_df14['dest_feature_pc1'].fillna(-1 ,inplace=True)
full_df14['dest_feature_pc2'].fillna(-1 ,inplace=True)
full_df14['dest_feature_pc3'].fillna(-1 ,inplace=True)

In [21]:
### missing values for duration_of_stay : 1
full_df14['duration_of_stay'].fillna(1 ,inplace=True)

In [23]:
full_df14['orig_destination_distance'].fillna(-1 ,inplace=True)

In [24]:
### Verify all the  missing values are replaced 
count_cols = full_df14.count(axis=0)
count_cols

srch_destination_id          26483412
hotel_cluster                26483412
user_id                      26483412
is_mobile                    26483412
is_package                   26483412
is_booking                   26483412
is_alone                     26483412
hotel_continent              26483412
hotel_country                26483412
hotel_market                 26483412
orig_destination_distance    26483412
srch_destination_type_id     26483412
srch_rm_cnt                  26483412
user_location_country        26483412
user_location_region         26483412
user_location_city           26483412
year                         26483412
month                        26483412
day                          26483412
hour                         26483412
part_of_day                  26483412
type_of_day                  26483412
season                       26483412
duration_of_stay             26483412
dest_feature_pc1             26483412
dest_feature_pc2             26483412
dest_feature

In [28]:
### Randomly select 10000 user_id
import random
unique_users = full_df14.user_id.unique()
sel_user_ids = [unique_users[i] for i in sorted(random.sample(range(len(unique_users)), 5000)) ]
rf_dat = full_df14[full_df14.user_id.isin(sel_user_ids)]

In [29]:
### downsampling 
rf_dat.shape

(121403, 27)

In [30]:
rf_dat.to_csv('/home/smita/RF_models/clean_sample_RF4.csv')

In [3]:
cleanSample =  pd.read_csv("/home/smita/RF_models/clean_sample_RF4.csv")

In [4]:
cleanSample.shape

(121403, 28)

In [5]:
unique_users = cleanSample.user_id.unique()

In [6]:
len(unique_users)

5000

In [7]:
## Due to memory limitation not able again resampling the data
### Randomly select 1000 user_id
import random
unique_users = cleanSample.user_id.unique()
sel_user_ids = [unique_users[i] for i in sorted(random.sample(range(len(unique_users)), 1000)) ]
rf_dat = cleanSample[cleanSample.user_id.isin(sel_user_ids)]

In [8]:
rf_dat.shape

(23731, 28)

In [9]:
rf_dat.to_csv('/home/smita/RF_models/clean_sample_RF4_23K.csv')

#### Using Features_using_hierarchial_clustering.R we added 4 more features

loading data from the file final_data_RF4_small.csv

In [10]:
new_dat = pd.read_csv('/home/smita/RF_models/final_data_RF4_small.csv')

In [12]:
new_dat.columns.values

array(['Unnamed: 0', 'X', 'Unnamed..0', 'srch_destination_id',
       'hotel_cluster', 'user_id', 'is_mobile', 'is_package', 'is_booking',
       'is_alone', 'hotel_continent', 'hotel_country', 'hotel_market',
       'orig_destination_distance', 'srch_destination_type_id',
       'srch_rm_cnt', 'user_location_country', 'user_location_region',
       'user_location_city', 'year', 'month', 'day', 'hour', 'part_of_day',
       'type_of_day', 'season', 'duration_of_stay', 'dest_feature_pc1',
       'dest_feature_pc2', 'dest_feature_pc3', 'cluster_id_srchDest',
       'cluster_id_hotel_cluster', 'cluster_id_UCRC', 'cluster_id_ULSrDest'], dtype=object)

In [13]:
### Remove index columns from the data : Unnamed: 0 
del new_dat['Unnamed: 0']
del new_dat['Unnamed..0']
del new_dat['X']

In [15]:
new_dat.shape

(23731, 31)

In [17]:
new_dat.iloc[1:3, 1:15]

,hotel_cluster,user_id,is_mobile,is_package,is_booking,is_alone,hotel_continent,hotel_country,hotel_market,orig_destination_distance,srch_destination_type_id,srch_rm_cnt,user_location_country,user_location_region
1,7,334385,1,0,0,1,2,50,416,180.1034,1,1,66,442
2,48,334385,1,0,0,1,2,50,416,164.2579,1,1,66,442


In [18]:
new_dat.iloc[1:3, 16:31]

,year,month,day,hour,part_of_day,type_of_day,season,duration_of_stay,dest_feature_pc1,dest_feature_pc2,dest_feature_pc3,cluster_id_srchDest,cluster_id_hotel_cluster,cluster_id_UCRC,cluster_id_ULSrDest
1,2014,6,10,14,evening,weekday,summer,1,-0.974259,-0.628523,-2.011131,1,1,1,1
2,2014,4,30,17,evening,weekday,spring,1,-0.974259,-0.628523,-2.011131,1,1,1,1


In [19]:
### encode the categorical variable to numberic before applying the RF
#part_of_day, type_of_day, season

from sklearn.preprocessing import LabelEncoder
var_mod = ['part_of_day', 'type_of_day', 'season']
le = LabelEncoder()
for i in var_mod:
    new_dat[i] = le.fit_transform(new_dat[i])


### Split the Data in Training and Test Set 

In [21]:
train = new_dat[new_dat.month < 8]
test = new_dat[new_dat.month >= 8]

In [22]:
train.shape

(11264, 31)

In [23]:
test.shape

(12467, 31)

### Remove Clicks from test set

In [24]:
test = test[test.is_booking == 1]

In [25]:
train.shape

(11264, 31)

In [26]:
test.shape

(946, 31)

#### Remove Unnecessory variables from dataframe

In [27]:
# Exclude Target and Predictors
predictor_set1 = [c for c in train.columns if c not in ['hotel_cluster', 'user_id']]

### Model -1

In [28]:
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier

model_RF = RandomForestClassifier(n_estimators=5, min_weight_fraction_leaf=0.1)
scores = cross_validation.cross_val_score(model_RF, train[predictor_set1], train['hotel_cluster'], cv=5)
scores

array([ 0.05237887,  0.04889868,  0.05908485,  0.07200358,  0.07490975])

In [29]:
model_RF.fit(train[predictor_set1], train['hotel_cluster'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.1, n_estimators=5, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### use model to predict outcome for test dataset

In [31]:
test_score = model_RF.score(test[predictor_set1], test['hotel_cluster'])
test_score

0.071881606765327691

#### Feature importance

In [33]:
import numpy as np
importance = model_RF.feature_importances_
std = np.std([model_RF.feature_importances_ for tree in model_RF.estimators_], axis=0)
indices = np.argsort(importance)[::-1]
print ("Feature sorted by their scores:")
df = sorted(zip(map(lambda x:round(x,4), model_RF.feature_importances_), predictor_set1), reverse=True)
imp = pd.DataFrame(df)
imp.columns = ['Score', 'Variable']
imp

Feature sorted by their scores:


,Score,Variable
0,0.3086,dest_feature_pc1
1,0.1631,dest_feature_pc2
2,0.1060,hotel_continent
3,0.0928,hotel_country
4,0.0852,hotel_market
5,0.0727,cluster_id_hotel_cluster
6,0.0576,srch_destination_id
7,0.0358,dest_feature_pc3
8,0.0249,orig_destination_distance
9,0.0224,duration_of_stay


In [34]:
### Model -2

In [39]:
predictor_set1

['srch_destination_id',
 'is_mobile',
 'is_package',
 'is_booking',
 'is_alone',
 'hotel_continent',
 'hotel_country',
 'hotel_market',
 'orig_destination_distance',
 'srch_destination_type_id',
 'srch_rm_cnt',
 'user_location_country',
 'user_location_region',
 'user_location_city',
 'year',
 'month',
 'day',
 'hour',
 'part_of_day',
 'type_of_day',
 'season',
 'duration_of_stay',
 'dest_feature_pc1',
 'dest_feature_pc2',
 'dest_feature_pc3',
 'cluster_id_srchDest',
 'cluster_id_hotel_cluster',
 'cluster_id_UCRC',
 'cluster_id_ULSrDest']

In [41]:
# Exclude Target and Predictors
predictor_set2 = imp.Variable[0:10]
predictor_set2

0             dest_feature_pc1
1             dest_feature_pc2
2              hotel_continent
3                hotel_country
4                 hotel_market
5     cluster_id_hotel_cluster
6          srch_destination_id
7             dest_feature_pc3
8    orig_destination_distance
9             duration_of_stay
Name: Variable, dtype: object

In [ ]:
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier

model_RF = RandomForestClassifier(n_estimators=5, min_weight_fraction_leaf=0.1)
scores = cross_validation.cross_val_score(model_RF, train[predictor_set2], train['hotel_cluster'], cv=5)

In [46]:
scores*100

array([ 5.1069402 ,  7.44493392,  6.97467792,  8.4078712 ,  7.26534296])

In [43]:
model_RF.fit(train[predictor_set2], train['hotel_cluster'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.1, n_estimators=5, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [45]:
test_score = model_RF.score(test[predictor_set2], test['hotel_cluster'])
test_score*100

8.2452431289640593